In [ ]:
import os
import myServices 
import randForReg as rfr
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("/Users/abdielfer/DESS/Internship2022/RNCanWork/datasetBasin1.csv")
# df=pd.to_csv('./teatDF.csv')

In [9]:
print(df.head())

   percentage   disToRiv       TWI      TPI      FAcc  slope  elevation  \
0           0  589.61853  10.29305  0.03323  29.52681    NaN   73.69112   
1           0  587.72870  10.76732  0.01434  47.44482    NaN   73.55061   
2           0  585.87543  10.93453 -0.02701  56.07990    NaN   73.37184   
3           0  586.94122  10.12663  0.08641  25.00000    NaN   73.84685   
4           0  585.00000  10.55997  0.03105  38.55986    NaN   73.66492   

   x_coord  y_coord  
0   370033  5267983  
1   370038  5267983  
2   370043  5267983  
3   370028  5267978  
4   370033  5267978  


In [20]:
df=teatDF.to_csv('./teatDF.csv')

In [ ]:
# to compute ececution time do:
# with timeit():
#     # your code, e.g., 
class timeit(): 
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

## Random Forest Section

In [ ]:
#Data import and exploration
datasetPath = ''
rfReg  = rfr.RandomForestRegression(datasetPath,'porcentage')
x_train,x_validation,y_train, y_validation = rfReg.getSplitedDataset()
rfr.printDataBalace(x_train, x_validation, y_train, y_validation,'porcentage')